# IMPORTACIÓN DE LIBRERÍAS

In [1]:
import glob
import pandas as pd
import xml.etree.ElementTree as ET
from datetime import datetime
import requests
from zipfile import ZipFile

# DESCARGA DE ARCHIVOS

In [2]:
def get_data():
    # Enlace de descarga de archivo
    !wget http://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip
    # Descomprimir el archivo
    with ZipFile('datasource.zip', 'r') as zip:
        zip.extractall('dealership_data')
        print('Archivo descomprimido en el directorio dealership_data')

# EXTRAER

In [3]:
#Función de extracción CSV
def extract_from_csv(file_to_process): 
    dataframe = pd.read_csv(file_to_process) 
    return dataframe

#Función de extracción JSON
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process, lines=True)
    return dataframe

#Función de extracción XML
def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel'])
    tree = ET.parse(file_to_process) 
    root = tree.getroot() 
    for person in root: 
        car_model = person.find("car_model").text 
        year_of_manufacture = int(person.find("year_of_manufacture").text)
        price = float(person.find("price").text) 
        fuel = person.find("fuel").text 
        dataframe = dataframe.append({"car_model":car_model, 
                                      "year_of_manufacture":year_of_manufacture, 
                                      "price":price, 
                                      "fuel":fuel}, 
                                     ignore_index=True) 
    return dataframe

## Función extraer

In [4]:
def extract():
    extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel']) 
    #Para archivos csv
    for csvfile in glob.glob("dealership_data/*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
    #Para archivos json
    for jsonfile in glob.glob("dealership_data/*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    #Para archivos xml
    for xmlfile in glob.glob("dealership_data/*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
    return extracted_data

# TRANSFORMAR

In [5]:
def transform(data):
    data['price'] = round(data.price, 2)
    return data

# CARGAR

In [6]:
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile)

## Ruta para los archivos de destino

In [7]:
tmpfile    = "dealership_temp.tmp"               # Almacenar todos los datos extraídos
logfile    = "dealership_logfile.txt"            # Todos los registros de eventos se almacenarán
targetfile = "dealership_transformed_data.csv"   # Los datos transformados se almacenan

## Registro de información actual

In [8]:
def log(message):
    timestamp_format = '%H:%M:%S-%h-%d-%Y'
    #Hour-Minute-Second-MonthName-Day-Year
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("dealership_logfile.txt","a") as f: f.write(timestamp + ',\n' + message + ' \n')

### Inicio de proceso ETL

In [9]:
log("Trabajo ETL iniciado")
log("Descargar datos de autos usados")
get_data()

--2022-01-12 22:59:03--  http://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4249 (4.1K) [application/zip]
Saving to: ‘datasource.zip’

datasource.zip      100%[===================>]   4.15K  --.-KB/s    in 0s      

2022-01-12 22:59:04 (123 MB/s) - ‘datasource.zip’ saved [4249/4249]

Archivo descomprimido en el directorio dealership_data


###  Inicio y finalización de fase de Extracción

In [10]:
log("Fase de extracción iniciada")
extracted_data = extract() 
log("Fase de extracción finalizada")

### Inicio y finalización de fase de Transformación

In [11]:
log("Fase de transformación iniciada")
transformed_data = transform(extracted_data)
log("Fase de transformación finalizada")

### Inicio y finalización de fase de Carga

In [12]:
log("Fase de carga iniciada")
load(targetfile,transformed_data)
log("Fase de carga finalizada")

### Finalización de ciclo ETL

In [13]:
log("Trabajo ETL finalizado")